In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import xgboost as xgb
# import catboost as ct
import seaborn as sns
from sklearn import ensemble,cross_validation,preprocessing
path='C:/Projects/Hackerearth_3/'
solution_path=path+'solutions'
%matplotlib inline
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn import metrics
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn import  preprocessing
from sklearn.manifold import TSNE
from scipy.spatial.distance import cdist
import numpy as np
import pandas as pd
from collections import Counter

from tqdm import tnrange

In [5]:
test_data=pd.read_csv(path+'test.csv')

In [6]:

data=pd.read_csv(path+'train.csv',sep='|',header=None,dtype={1:object})
movies_data=pd.read_csv(path+'movies_meta.csv',sep='|')

In [7]:
data.columns=['user_id','Title','watch_time']
# movies_data.columns=['title','Genre','Language','Duration']


In [8]:
data.head()

,user_id,Title,watch_time
0,0,avan ivan,6139.0
1,0,janatha garage,6706.0
2,1,besh korechi prem korechi,1229.0
3,2,sankeerthana,2633.0
4,3,baahubali - the beginning,967.0


In [9]:
np.unique(data['user_id'])

array([     0,      1,      2, ..., 876594, 876595, 876596], dtype=int64)

In [10]:
movies_data.head()

,Title,Genre,Language,Duration
0,dil toh baccha hai ji,romance,hindi,8185.0
1,golmaal: fun unlimited,comedy,hindi,8554.0
2,fukrey,comedy,hindi,7983.0
3,dabangg 2,action,hindi,7104.0
4,bhaag milkha bhaag,drama,hindi,10689.0


In [11]:
new_data=pd.merge(data,movies_data,on='Title',how='left')

In [12]:
new_data['ratings']=((new_data['watch_time']/new_data['Duration'])*10).astype(int)

In [13]:
new_data['ratings']=np.minimum(new_data['ratings'],10)

In [14]:

new_data['aff']=new_data['watch_time']/new_data['Duration']
new_data.head()

,user_id,Title,watch_time,Genre,Language,Duration,ratings,aff
0,0,avan ivan,6139.0,comedy,tamil,7860.0,7,0.781043
1,0,janatha garage,6706.0,action,"telugu,malayalam,malayalam",9066.0,7,0.739687
2,1,besh korechi prem korechi,1229.0,romance,bengali,8614.0,1,0.142675
3,2,sankeerthana,2633.0,romance,telugu,8275.0,3,0.318187
4,3,baahubali - the beginning,967.0,action,telugu,9158.0,1,0.105591


In [15]:
user_mat=new_data.groupby('user_id').agg({'Title':'count','Duration':'mean','aff':'mean'}).reset_index()

In [16]:
user_mat.head()

,user_id,Duration,aff,Title
0,0,8463.0,0.760365,2
1,1,8614.0,0.142675,1
2,2,8275.0,0.318187,1
3,3,9158.0,0.105591,1
4,4,7099.6,0.431187,5


In [17]:
user_mat.columns[1:]

Index([u'Duration', u'aff', u'Title'], dtype='object')

In [18]:
normalizer = preprocessing.Normalizer()
X1 = normalizer.fit_transform(user_mat[user_mat.columns[1:]]) 

In [ ]:
# k means determine k
distortions = []
# silhouette_dist = []
K = range(2,11)
for idx in tnrange(len(K)):
    k = K[idx]
    kmeans = KMeans(n_clusters=k, init='k-means++', max_iter=300, random_state=777)
    kmeans = kmeans.fit(X1)
    cluster_labels = kmeans.predict(X1)
    distortions.append(sum(np.min(cdist(X1, kmeans.cluster_centers_, 'euclidean'), axis=1)) / X1.shape[0])
#     sil_scores = []
#     for i in range(6):
#         sil_scores.append(silhouette_score(X1, cluster_labels, sample_size=10000))
#     silhouette_dist.append(np.mean(sil_scores))

In [ ]:
fig = plt.figure(figsize=(10,8))
plt.plot(K, distortions, label='distortions', lw=2, marker='o')
# plt.plot(K, silhouette_dist, label='silhouette dist', lw=2, marker='s')
plt.xlabel('number of clusters')
plt.ylabel('Distortion/silhouette distance')
plt.title('The Elbow Method showing the optimal k')
plt.grid()
plt.legend(loc='upper right')

# plt.gcf().autofmt_xdate()

plt.show()

In [69]:
kmeans = KMeans(n_clusters= 7, init='k-means++', max_iter=300,  n_jobs= -1, random_state=0)
kmeans.fit(X1)

centroids = kmeans.cluster_centers_
labels = kmeans.labels_


In [73]:
user_mat['cluster']=labels

In [79]:
test_data['val']=1

In [82]:
new_data=pd.merge(pd.merge(new_data,user_mat[['user_id','cluster']],on='user_id'),test_data,on='user_id',how='left').fillna(0)

In [84]:
clus1=new_data[new_data['cluster']==1]

In [86]:
target=clus1[clus1['val']==1]

In [97]:
len(np.unique(target['user_id']))

789

In [123]:
base_user=clus1[clus1['val']==0]

In [124]:
len(np.unique(base_user['user_id']))

32672

In [128]:
import random
sam=5000

In [133]:
base_user_list=np.array(random.sample(np.unique(base_user['user_id']).tolist(),sam)).astype(int)

In [135]:
base_user2=base_user[base_user['user_id'].isin(base_user_list)]

In [137]:
final_data=pd.concat([target,base_user2])

In [138]:
final_data.shape

(47643, 10)

In [140]:
final_data.head(2)

,user_id,Title,watch_time,Genre,Language,Duration,ratings,aff,cluster,val
3093,1368,bedroom partner,1228.0,drama,hindi,4252.0,2,0.288805,1,1.0
3094,1368,daddy cool,8817.0,comedy,hindi,6099.0,10,1.445647,1,1.0


In [141]:
matri=pd.pivot_table(final_data,columns='Title',index='user_id',values='ratings',fill_value=0)

In [143]:
Q = matri.values

In [144]:
W = Q>0.5
W[W == True] = 1
W[W == False] = 0
# To be consistent with our Q matrix
W = W.astype(np.float64, copy=False)

In [145]:
lambda_ = 0.1
n_factors = 100
m, n = Q.shape
n_iterations = 20

In [146]:
X = 5 * np.random.rand(m, n_factors) 
Y = 5 * np.random.rand(n_factors, n)

In [147]:
def get_error(Q, X, Y, W):
    return np.sum((W * (Q - np.dot(X, Y)))**2)

In [149]:
def print_recommendations(W, Q, Q_hat, movie_titles):
    #Q_hat -= np.min(Q_hat)
    #Q_hat[Q_hat < 1] *= 5
    Q_hat -= np.min(Q_hat)
    Q_hat *= float(5) / np.max(Q_hat)
    movie_ids = np.argmax(Q_hat - 5 * W, axis=1)
    for jj, movie_id in zip(range(m), movie_ids):
        #if Q_hat[jj, movie_id] < 0.1: continue
        print('User {} liked {}\n'.format(jj + 1, ', '.join([movie_titles[ii] for ii, qq in enumerate(Q[jj]) if qq > 3])))
        print('User {} did not like {}\n'.format(jj + 1, ', '.join([movie_titles[ii] for ii, qq in enumerate(Q[jj]) if qq < 3 and qq != 0])))
        print('\n User {} recommended movie is {} - with predicted rating: {}'.format(
                    jj + 1, movie_titles[movie_id], Q_hat[jj, movie_id]))
        print('\n' + 100 *  '-' + '\n')

In [ ]:
weighted_errors = []
for ii in range(n_iterations):
    for u, Wu in enumerate(W):
        X[u] = np.linalg.solve(np.dot(Y, np.dot(np.diag(Wu), Y.T)) + lambda_ * np.eye(n_factors),
                               np.dot(Y, np.dot(np.diag(Wu), Q[u].T))).T
    for i, Wi in enumerate(W.T):
        Y[:,i] = np.linalg.solve(np.dot(X.T, np.dot(np.diag(Wi), X)) + lambda_ * np.eye(n_factors),
                                 np.dot(X.T, np.dot(np.diag(Wi), Q[:, i])))
    weighted_errors.append(get_error(Q, X, Y, W))
    print('{}th iteration is completed'.format(ii))
weighted_Q_hat = np.dot(X,Y)

In [ ]:
print_recommendations(Q_hat=weighted_Q_hat)